## [Kaggle: Consumer Complaint Database](https://www.kaggle.com/selener/consumer-complaint-database)

- *Download the rows.csv file & save to ../datasets directory before proceeding further*

This Notebook is used to parse the original dataset & prepare a subset of this dataset by compacting some of the product types & retaining 5 product types.

In [23]:
import pandas as pd

In [24]:
original_dataset = pd.read_csv('rows.csv')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (4,5,6,11,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
original_dataset.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/10/2019,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,328XX,Older American,NaN,Web,05/10/2019,In progress,Yes,NaN,3238275
1,05/10/2019,Checking or savings account,Other banking product or service,Managing an account,Deposits and withdrawals,NaN,NaN,BOEING EMPLOYEES CREDIT UNION,WA,98204,NaN,NaN,Referral,05/10/2019,Closed with explanation,Yes,NaN,3238228
2,05/10/2019,Debt collection,Payday loan debt,Communication tactics,Frequent or repeated calls,NaN,NaN,CURO Intermediate Holdings,TX,751XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237964
3,05/10/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Old information reappears or never goes away,NaN,NaN,Ad Astra Recovery Services Inc,LA,708XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3238479
4,05/10/2019,Checking or savings account,Checking account,Managing an account,Banking errors,NaN,NaN,ALLY FINANCIAL INC.,AZ,85205,NaN,NaN,Postal mail,05/10/2019,In progress,Yes,NaN,3238460
5,05/10/2019,Mortgage,Other type of mortgage,Closing on a mortgage,NaN,NaN,NaN,Statebridge Company,NJ,08302,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237885
6,05/10/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,"Diversified Consultants, Inc.",TX,NaN,Servicemember,NaN,Web,05/10/2019,In progress,Yes,NaN,3238585
7,05/10/2019,Student loan,Federal student loan servicing,Struggling to repay your loan,Problem lowering your monthly payments,NaN,NaN,Student Loan Direct,TX,773XX,Older American,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3238221
8,05/10/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,NaN,NaN,"Diversified Consultants, Inc.",SC,296XX,Servicemember,NaN,Web,05/10/2019,In progress,Yes,NaN,3238545
9,05/10/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,CONTRACT CALLERS INC,MD,20774,NaN,NaN,Web,05/10/2019,In progress,Yes,NaN,3238458


In [26]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282355 entries, 0 to 1282354
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Date received                 1282355 non-null  object
 1   Product                       1282355 non-null  object
 2   Sub-product                   1047189 non-null  object
 3   Issue                         1282355 non-null  object
 4   Sub-issue                     751169 non-null   object
 5   Consumer complaint narrative  383564 non-null   object
 6   Company public response       449082 non-null   object
 7   Company                       1282355 non-null  object
 8   State                         1262955 non-null  object
 9   ZIP code                      1167057 non-null  object
 10  Tags                          175643 non-null   object
 11  Consumer consent provided?    690654 non-null   object
 12  Submitted via                 1282355 non-

In [27]:
print('Number of Unique Products:', original_dataset.Product.nunique())

Number of Unique Products: 18


In [28]:
original_dataset.Product.unique()

array(['Checking or savings account', 'Debt collection',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Mortgage', 'Student loan', 'Vehicle loan or lease',
       'Credit card or prepaid card',
       'Payday loan, title loan, or personal loan',
       'Money transfer, virtual currency, or money service',
       'Credit reporting', 'Credit card', 'Bank account or service',
       'Consumer Loan', 'Prepaid card', 'Other financial service',
       'Payday loan', 'Money transfers', 'Virtual currency'], dtype=object)

In [29]:
print('Rows with Null Consumer Complaints: ',
      original_dataset[original_dataset['Consumer complaint narrative'].isna()].shape[0])

Rows with Null Consumer Complaints:  898791


In [30]:
print('Original number of rows:', original_dataset.shape[0])
original_dataset.dropna(axis=0, how='any', subset=['Consumer complaint narrative'], inplace=True)
print('Remaining number of rows:', original_dataset.shape[0])

Original number of rows: 1282355
Remaining number of rows: 383564


In [31]:
original_dataset.groupby(['Product'])\
    [['Consumer complaint narrative']]\
    .count()\
    .rename({'Consumer complaint narrative': 'Count'}, axis=1)\
    .sort_values('Count', ascending=False)

,Count
Product,
"Credit reporting, credit repair services, or other personal consumer reports",92378
Debt collection,86710
Mortgage,52987
Credit reporting,31588
Student loan,21810
Credit card or prepaid card,21379
Credit card,18838
Bank account or service,14885
Checking or savings account,12881


In [32]:
# Replace Repetitive Product Types. This is to ensure that data is compacted and doesnt contain overlapping classes.

original_dataset.replace(
    {'Product': 
     {'Credit reporting, credit repair services, or other personal consumer reports': 
          'Credit reporting, repair, or other', 
      'Credit reporting': 'Credit reporting, repair, or other',
      'Credit card': 'Credit card or prepaid card',
      'Prepaid card': 'Credit card or prepaid card',
      'Payday loan': 'Payday loan, title loan, or personal loan',
      'Money transfers': 'Money transfer, virtual currency, or money service',
      'Virtual currency': 'Money transfer, virtual currency, or money service'}},
    inplace=True)

In [33]:
original_dataset.groupby(['Product'])\
    [['Consumer complaint narrative']]\
    .count()\
    .rename({'Consumer complaint narrative': 'Count'}, axis=1)\
    .nlargest(20, 'Count')

,Count
Product,
"Credit reporting, repair, or other",123966
Debt collection,86710
Mortgage,52987
Credit card or prepaid card,41667
Student loan,21810
Bank account or service,14885
Checking or savings account,12881
Consumer Loan,9474
"Money transfer, virtual currency, or money service",6979


In [34]:
# Select Different Classes which has some good amount of data & doesnt overlap much.
# This is to keep the problem statement simple.
product_types = [
    'Credit reporting, repair, or other',
    'Debt collection',
    'Student loan',
    'Bank account or service',
    'Money transfer, virtual currency, or money service'
]
sub_dataset = original_dataset[original_dataset['Product'].isin(product_types)].copy()
sub_dataset.shape

(254350, 18)

In [35]:
sub_dataset.groupby(['Product'])[['Consumer complaint narrative']]\
    .count()\
    .rename({'Consumer complaint narrative': 'Count'}, axis=1)\
    .sort_values('Count', ascending=False)

,Count
Product,
"Credit reporting, repair, or other",123966
Debt collection,86710
Student loan,21810
Bank account or service,14885
"Money transfer, virtual currency, or money service",6979


In [36]:
sub_dataset.rename(columns={'Consumer complaint narrative': 'Complaint_text'}, 
                   inplace=True)

In [37]:
sub_dataset.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Complaint_text,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
29904,03/23/2019,"Credit reporting, repair, or other",Credit reporting,Incorrect information on your report,Account information incorrect,The Summer of XX/XX/2018 I was denied a mortga...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",IL,NaN,NaN,Consent provided,Web,03/23/2019,Closed with explanation,Yes,NaN,3189109
30629,03/22/2019,"Credit reporting, repair, or other",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",VA,220XX,NaN,Consent provided,Web,03/22/2019,Closed with explanation,Yes,NaN,3187982
30735,03/22/2019,"Credit reporting, repair, or other",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,770XX,NaN,Consent provided,Web,03/22/2019,Closed with explanation,Yes,NaN,3187954
30795,03/22/2019,"Credit reporting, repair, or other",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TX,787XX,NaN,Consent provided,Web,03/22/2019,Closed with explanation,Yes,NaN,3188091
30807,03/22/2019,"Credit reporting, repair, or other",Credit reporting,Incorrect information on your report,Account information incorrect,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,951XX,NaN,Consent provided,Web,03/22/2019,Closed with explanation,Yes,NaN,3188119


In [42]:
training_dataset_indexes = []
for product in sub_dataset['Product'].unique():
    indexes = sub_dataset[sub_dataset.Product == product]\
    .sample(4000, random_state=19).index
    training_dataset_indexes.extend(indexes)
    
training_dataset = sub_dataset.loc[training_dataset_indexes][['Product', 'Complaint_text']].copy()
training_dataset.shape

(20000, 2)

In [46]:
sub_dataset.drop(index=training_dataset_indexes, inplace=True)

In [49]:
test_dataset_indexes = []
for product in sub_dataset['Product'].unique():
    indexes = sub_dataset[sub_dataset.Product == product]\
    .sample(1000, random_state=19).index
    test_dataset_indexes.extend(indexes)
    
test_dataset = sub_dataset.loc[test_dataset_indexes][['Product', 'Complaint_text']].copy()
test_dataset.shape

(5000, 2)

In [51]:
training_dataset.to_csv('../datasets/consumer_complaints_training_dataset.csv', index=False)
test_dataset.to_csv('../datasets/consumer_complaints_test_dataset.csv', index=False)